# Neural Nets

In this part of the laboratory, you are to complete the API for a Neural Net. Then afterwards, you are to construct various neural nets using the API to solve abstract learning problems.

## Completing the implementation

We have provided you a skeleton of the Neural Net in the Neural net code. You are to complete the unimplemented methods.

The three classes `Input`, `PerformanceElem`, and `Neuron` all have incomplete implementation of the following two functions:
     
    def output(self)
    def dOutdX(self, elem)
Your first task is to fill in all 6 functions to complete the API.

## Output

The function `output(self)` produces the output of each of these elements.

Be sure to use the sigmoid and ds/dz functions as discussed in the course:

    o = s(z) = 1.0 / (1.0 + e**(-z))
    ds(o)/dz = s(z) * (1 - s(z)) = o * (1 - o)
and the performance function and its derivative as discussed in the course:

    P(o) = -0.5 (d - o)**2
    dP(o)/dx = (d - o)
    
## Derivatives

The function `dOutdX(self, elem)` generates the value of the partial derivative with respect to a given weight element.

Recall, neural nets update a given weight by computing the partial derivative of the performance function with respect to that weight. The formula we have used in class is as follows:

    wi' = wi + rate * dP / dwi
    
In our code this is represented as (see `def train()` -- you don't have to implement this):

    w.set_next_value( w.get_value() + rate * network.performance.dOutdX(w) )
The element passed to the `dOutdX` function is always a weight. Namely it is the weight that we are doing the partial over. Your job is to figure out how to define `dOutdX()` in terms of recursively calling `dOutdX()` or `output()` over the inputs and weights of a network element.

For example, consider the Performance element P, `P.dOutdX(w)` could be defined in the following recursive fashion:

    dP/d(w) = dP/do * do/dw # applying the chain rule
            = (d-o) * o.dOutdX(w)
Here `o` is the output of the Neuron that is directly connected to P.

For Neuron units, there would be more cases to consider. Namely,


1.   The termination case where the weight being differentiated over is one of the (direct) weights of the current neuron.
2.   The recursive case where the weight is not one that is directly connected (but is a descendant weight).

This implementation models the process of computing the chain of derivatives recursively. This top-down approach works from the output layer towards the input layers. This is in contrast to the more conventional approach (taught in the course) where you computed a per-node little-delta, and then recursively computed the weight updates bottom-up, from input towards output layers.

## About the API Classes

Most of the classes in the Neural Network inherit from the following two abstract classes:

### `ValuedElement`

This interface class allows an element to have a settable value. `Input` (e.g. i1, i2) and `Weight` (e.g. w1A, wAB) are subclasses of `ValueElement`.

Elements that are subclassed all have these methods:

*  `set_value(self,val)` - set the value of the element
*  `get_value(self)` - get the value of the element
*  `get_name(self)` - get the name of the element

### `DifferentiableElement`

This abstract class defines the interface for elements that have outputs and are involved in partial derivatives.

* `output(self):` returns the output of this element
* `dOutdX(self,elem):` returns the partial derivative with respect to another element

`Inputs`, `Neurons`, and `PerformanceElem` are the three subclasses that implement `DifferentiableElement`. You will have to complete the interface for these classes.

### `Weight(ValuedElement)`

Represents update-able weights in the network. In addition to `ValueElement` functions are the following methods, which are used for the training algorithm (you will not need them in your implementation):

* `set_next_value(self,val):` which sets the next weight value in `self.next_value`
* `update(self):` which sets the current weight to the value stored in `self.next_value`

### `Input(DifferentiableElement, ValuedElement)`

Represents inputs to the network. These may represent variable inputs as well as fixed inputs (i.e. threshold inputs) that are always set to -1. `output()` of `Input` units should simply return the value they are set to during training or testing.

**`dOutdX(self, elem)` of an `Input` unit should return 0, since there are no weights directly connected into inputs.**

### `Neuron(DifferentiableElement)`

Represents the actual neurons in the neural net. The definitions for `output` and `dOutdX` already contains some code. Namely, we've implemented a value caching mechanism to speed up the training / testing process. So instead of implementing output and `dOutdx` directly you should instead implement:

* `compute_output(self):`
* `compute_doutdx(self,elem):`

### `PerformanceElem(DifferentiableElement)`

Represents a Performance Element that allows you to set the desired output.

* `set_desired` which sets `my_desired_val`

To better understand back-propagation, you should take a look at the methods `train` and `test` in the Neural net code to see how everything is put together.

## Unit Testing

Once you've completed the missing functions, we have provided the Neural net tester code to help you unit test. You will need to pass the tests in this unit tester before you can move on to the next parts.

To check if your implementation works, run:

    neural_net_tester(["simple"])
This makes sure that your code works and can learn basic functions such as AND and OR.

## Building Neural Nets

Once you have finished implementing the Neural Net API, you will be tasked to build three networks to learn various abstract data sets.

Here is an example of how to construct a basic neural network:

    def make_neural_net_basic():
       """Returns a 1 neuron network with 2 variable inputs, and 1 fixed input."""
       i0 = Input('i0',-1.0) # this input is immutable
       i1 = Input('i1',0.0)
       i2 = Input('i2',0.0)
       
       w1A = Weight('w1A',1)
       w2A = Weight('w2A',1)
       wA = Weight('wA', 1)
       
       # the inputs must be in the same order as their associated weights
       A = Neuron('A', [i1,i2,i0], [w1A,w2A,wA])
       P = PerformanceElem(A, 0.0)
       
       # Package all the components into a network
       # First list the PerformanceElem P
       # Then list all neurons afterwards
       net = Network(P,[A])
       
       return net
       
## Naming conventions

IMPORTANT: Be sure to use the following naming convention when creating elements for your networks:

Inputs:

* Format: `'i'` + input_number
* Conventions:
   * Start numbering from 1.
   * Use the same `i0` for all the fixed -1 inputs.
* Examples: `'i1'`, `i2`.

Weights:
* Format `'w' + from_identifier + to_identifier`
* Examples:
   * `w1A` for weight from Input `i1` to Neuron `A`.
   * `wBC` for weight from Neuron `B` to Neuron `C`.
   
Neurons:

* Format: `alphabet_letter`.
* Convention: Assign neuron names in order of distance to the inputs.
* Example: `A` is the neuron closest to the inputs, and on the left-most (or top-most) side of the net.
* For ties, order neurons from left to right or top to bottom (depending on how you draw orient your network).

## Building a 2-layer Neural Net

Now use the Neural Net API you've just completed and tested to create a two layer network that looks like the following. 

![alt text](https://ai6034.mit.edu/wiki/images/NeuralNet.png)

Fill your answer in the function stub:
   
    def make_neural_net_two_layer()
in the Neural net code. 

Your 2-layer neural net should now be able to learn slightly harder datasets, such as the classic non-linearly separable examples such as NOT-EQUAL (XOR) and EQUAL.

When initializing the weights of the network, you should use random weights. To get deterministic random initial weights so that tests are reproducible you should first seed the random number generator, and then generate the random weights.

    seed_random()
    
    wt = random_weight()
    ...use wt...
    wt2 = random_weight()
    ...use wt2...
    
Note: the function `random_weight()` in the Neural net code uses the python function `random.randrange(-1,2)` to compute initial weights. This function generates values: -1, 0, 1 (randomly). While this may seem like a mistake, what we've found empirically is that this actually performs better than using `random.uniform(-1, 1)`. Be our guest and play around with the `random_weight` function. You'll find that Neural Nets can be quite sensitive to initialization weight settings. (Recall what happens if you set all weights to the same value.)    

To test your completed network, run:

    neural_net_tester(["two_layer"])
Your network should learn and classify all the datasets in the `harder_data_set` from the Neural net data code with 100% accuracy.

## Designing For More Challenging Datasets

Now it's your turn to design the network. We want to be able to classify more complex data sets.

Specifically we want you to design a new network that should theoretically be able to learn and classify the following datasets:

1. The letter-L.

       4 + -
       3 + -
       2 + -
       1 + - - - -
       0 - + + + +
         0 1 2 3 4
2. This moat-like shape:

       4 - - - - -
       3 -       -
       2 -   +   -
       1 -       -
       0 - - - - -
         0 1 2 3 4
3. This `patchy` shape:

       4 - -   + +
       3 - -   + +
       2
       1 + +   - -
       0 + +   - -
         0 1 2 3 4
         
We claim that a network architecture containing 5 neuron nodes or less can fully learn and classify all three shapes. In fact, we require it!

Construct a new network in:

    def make_neural_net_challenging()
that can (theoretically) perfectly learn and classify all three datasets.

To test your network on the first 2 of these shapes, run

    neural_net_tester(["challenging"])
To pass our tests, your network must get 100% accuracy within 10000 iterations.

Now try your architecture on the third dataset, patchy. Run:

    neural_net_tester(["patchy"])
Depending on your architecture and your initial weights, your network may either easily learn patchy or get stuck in a local maximum. Does your network completely learn the dataset within 10000 iterations? If not, take a look at the weights output at the end of the 10000 iterations. Plot the weights in terms of a linear function on a 2D graph. Do the boundaries tell you why there might be a local maximum?

## Manually Setting Weights

You can have your network learn the dataset `patchy` perfectly and very quickly if the proper weights are set.

You can use either of these strategies to determine the proper weights.
1. You can experimentally determine the right weights by running your network until it perfectly learns the dataset. You will probably need to increase the max-iterations parameter, or playing around with different initial weight settings.
2. You can try to solve for the weights analytically.

In either case, we want you to find preset weights to the same network that you built in the last part. Your new weight-preset network should be able to learn the `patchy` problem with only 1000 additional iterations of training.

After you've found the optimal weights, fill in:

    def make_neural_net_with_weights()
To test, run:
    
    neural_net_tester(["weights"])
    
If everything tests with an accuracy of 1.0, then you've completed the Neural Networks portion of the laboratory.

# Boosting

You're still trying to use AI to predict the votes of politicians. ID-Trees were great, but you've heard about these other magnificent learning algorithms like SVMs and Boosting. Boosting sounds easier to implement and had a pretty good reputation, so you decide to start with that.

To make sure that you interpret the results without letting your political preconceptions get in the way, you dig up some old data to work with: in particular, the data from the 4th House of Representatives, which met from 1796 to 1797. (This is when the two-party system first emerged, with the two parties being designated "Federalists" and "Republicans".)

You experiment with that data before going on to the 2007-2008 data, finding that Congressmen in 1796 were much more clear about what they were voting on than in 2008.

The framework for a boosting classifier can be found in the Boost code. You need to finish coding it, and then use it to learn some classifiers and answer a few questions.

## A (clever|cheap) trick

The boosting code uses a trick that means it only has to try half the number of base classifiers.

It turns out that AdaBoost does not really care which side of its base classifier is +1 and which side is -1. If you choose a classifier that is the opposite of the best classifier -- it returns -1 for most points that should be +1, and returns +1 for most points that should be -1, and therefore has a high error rate -- it works the same as if you had chosen the negation of that classifier, which is the best classifier.

If the data reweighting step is implemented correctly, it will produce the same weights given a classifier or its opposite. Also, a classifier with a high error rate will end up with a *negative* alpha value, so that in the final "vote" of classifiers it will act like its opposite. So the important thing about a classifier isn't that its error rate is *low* -- it's that the error rate is *far from 1/2*.

In the boosting code, we take advantage of this. We include only classifiers that output +1 for voting YES on a particular motion, or +1 for voting NO on a particular motion, and as the "best classifier" we choose the classifier whose error rate is *farthest from 1/2*. If the error rate is high, then the result will act like a classifier that outputs +1 for "voting NO or abstaining", or +1 for "voting YES or abstaining", respectively. This means we don't have to include these classifiers in the base classifier list, speeding up the algorithm by a factor of 2.

## Completing the code

Here are the parts that you need to complete:
* In the `BoostClassifier` class, the `classify` method is also undefined. Define it so that you can use a trained `BoostClassifier` as a classifier, outputting +1 or -1 based on the weighted results of its base classifiers.
* In the `BoostClassifier` class in the Boost code, the `update_weights` method is undefined. You need to define this method so that it changes the data weights in the way prescribed by the AdaBoost algorithm. Note: There are two ways of implementing this update; they happen to be mathematically equivalent.)
* In the To be implemented code, the `most_misclassified` function is undefined. You will need to define it to answer the questions.

**Remember to use the supplied `legislator_info(datum)` to output your list of the most-misclassified data points!**

## Questions

Answer the two questions `republican_newspaper_vote` and `republican_sunset_vote` in the To be implemeted code.

When you are asked how a particular political party would vote on a particular motion, disregard the possibility of abstaining. If your classifier results indicate that the party *wouldn't* vote NO, consider that an indication that the party would vote YES.

# Hints

## Neural Nets

If you are having problems with getting your network to convergence on certain problems, try the following:
1. Order your weight initialization (i.e. calls to `random_weight()`) from the bottom-most weights to the top-most weights in you network. While this ordering theoretically irrelevant, we've found that this ordering worked well in practice (in conjunction with 1 above). NN are unfortunately quite sensitive to initial weight settings.
2. Play around with the `seed_random` function to try different starting random seeds, although seeding the random function with 0 is what worked for us.
3. If none of these work, try setting weights that are close to what you want in terms of the final expected solution.

## FAQs

**Q:** I can't figure out how to write `compute_doutdx` for Neurons.

**A:** Here's a more step-by-step explanation of what this function does:

So you have a neural net, and you're at a particular neuron, call it D. This neuron D has some inputs - for example, maybe it has an input y_B from neuron B with weight w_BD, and an input y_C from neuron C with weight w_CD, and a constant input (-1) with weight w_D. So its total input - let's call it "z" - is (y_B\*w_BD + y_C\*w_CD + -1\*w_D). It's going to put that input z through its sigmoid function to get its output - let's call that y_D.

Now, we want to compute the derivative of y_D with respect to some weight w. There are two cases.

In the first case, w is one of the weights on something that goes directly into D - that is, w is w_BD or w_CD or w_D. In that case, we just do chain rule once, and we see that

d(y_D)/d(w) = d(y_D)/dz * dz/dw.

We know the derivative of the sigmoid function, so d(y_D)/dz is y_D\*(1-y_D). And the derivative of z with respect to w is just whatever thing that weight w was being multiplied by - so for example, if w was w_BD, then dz/dw would be y_B.

In the second case, w is not one of the weights going directly into D, it's probably something in an earlier layer, so we have to do some more chain rule to find dz/dw. We know z is (y_B\*w_BD + y_C\*w_CD + -1\*w_D) (because we defined it to be that), so to find dz/dw, we can find the derivative of each of those terms:

dz/dw = w_BD\*(d(y_B)/dw) + w_CD\*(d(y_C)/dw) + w_D\*(d(-1)/dw).

And how do we get the d(y_B)/dw etc? Well, y_B is the output of neuron B, so that's just `B.dOutdX(w)`. However, w might not have come from any path through B, so we only want to include the terms for which w is actually relevant. For that we can use the `isa_descendant_weight_of function`.

**Q:** How do I figure out how the boost classifier thinks a Republican would vote on the newspapers thing?

**A:** The classifiers are supposed to return +1 for Republican and -1 for anything else (there's a comment right above the definition of `BaseVoteClassifier` in the Boost code that says this). `boost_1796.classifiers` is a list of the weak classifiers and their corresponding weights in the trained classifier. You can look through those and find the newspaper one, and look at whether the weights on it are positive or negative.

**Q:** How can I tell which input goes with which weight, in the `my_inputs` and `my_weights` lists in a Neuron?

**A:** The two lists are in the same order - `my_weights[0]` is the weight for `my_inputs[0]`, etc.

**Q:** I can't get my "challenging" neural net to correctly recognize all of the data sets. What can I do?

**A:** Make sure you read the hints. You'll just need to play around with the initial weights.




## Neural net

In [ ]:
# - In this file we have an incomplete skeleton of
# a neural network implementation.  Follow the instructions
# and complete the NotImplemented methods below.
#

import math
import random
from functools import cmp_to_key

class ValuedElement(object):
    """
    This is an abstract class that all Network elements inherit from
    """
    def __init__(self,name,val):
        self.my_name = name
        self.my_value = val

    def set_value(self,val):
        self.my_value = val

    def get_value(self):
        return self.my_value

    def get_name(self):
        return self.my_name

    def __repr__(self):
        return "%s(%s)" %(self.my_name, self.my_value)

class DifferentiableElement(object):
    """
    This is an abstract interface class implemented by all Network
    parts that require some differentiable element.
    """
    def output(self):
        raise NotImplementedError("This is an abstract method")

    def dOutdX(self, elem):
        raise NotImplementedError("This is an abstract method")

    def clear_cache(self):
        """clears any precalculated cached value"""
        pass

class Input(ValuedElement,DifferentiableElement):
    """
    Representation of an Input into the network.
    These may represent variable inputs as well as fixed inputs
    (Thresholds) that are always set to -1.
    """
    def __init__(self,name,val):
        ValuedElement.__init__(self,name,val)
        DifferentiableElement.__init__(self)

    def output(self):
        """
        Returns the output of this Input node.
        
        returns: number (float or int)
        """
        return self.get_value();

    def dOutdX(self, elem):
        """
        Returns the derivative of this Input node with respect to 
        elem.

        elem: an instance of Weight

        returns: number (float or int)
        """
        return 0;

class Weight(ValuedElement):
    """
    Representation of an weight into a Neural Unit.
    """
    def __init__(self,name,val):
        ValuedElement.__init__(self,name,val)
        self.next_value = None

    def set_next_value(self,val):
        self.next_value = val

    def update(self):
        self.my_value = self.next_value

class Neuron(DifferentiableElement):
    """
    Representation of a single sigmoid Neural Unit.
    """
    def __init__(self, name, inputs, input_weights, use_cache=True):
        assert len(inputs)==len(input_weights)
        for i in range(len(inputs)):
            assert isinstance(inputs[i],(Neuron,Input))
            assert isinstance(input_weights[i],Weight)
        DifferentiableElement.__init__(self)
        self.my_name = name
        self.my_inputs = inputs # list of Neuron or Input instances
        self.my_weights = input_weights # list of Weight instances
        self.use_cache = use_cache
        self.clear_cache()
        self.my_descendant_weights = None

    def get_descendant_weights(self):
        """
        Returns a mapping of the names of direct weights into this neuron,
        to all descendant weights.
        """
        if self.my_descendant_weights is None:
            self.my_descendant_weights = {}
            inputs = self.get_inputs()
            weights = self.get_weights()
            for i in range(len(weights)):
                weight = weights[i]
                weight_name = weight.get_name()
                self.my_descendant_weights[weight_name] = set()
                input = inputs[i]
                if not isinstance(input, Input):
                    descendants = input.get_descendant_weights()
                    for name, s in descendants.items():
                        st = self.my_descendant_weights[weight_name]
                        st = st.union(s)
                        st.add(name)
                        self.my_descendant_weights[weight_name] = st

        return self.my_descendant_weights

    def isa_descendant_weight_of(self, target, weight):
        """
        Checks if [target] is a indirect input weight into this Neuron
        via the direct input weight [weight].
        """
        weights = self.get_descendant_weights()
        if weight.get_name() in weights:
            return target.get_name() in weights[weight.get_name()]
        else:
            raise Exception("weight %s is not connect to this node: %s"
                            %(weight, self))

    def has_weight(self, weight):
        """
        Checks if [weight] is a direct input weight into this Neuron.
        """
        weights = self.get_descendant_weights()
        return weight.get_name() in self.get_descendant_weights()

    def get_weight_nodes(self):
        return self.my_weights

    def clear_cache(self):
        self.my_output = None
        self.my_doutdx = {}

    def output(self):
        # Implement compute_output instead!!
        if self.use_cache:
            # caching optimization, saves previously computed dOutDx.
            if self.my_output is None:
                self.my_output = self.compute_output()
            return self.my_output
        return self.compute_output()

    def compute_output(self):
        """
        Returns the output of this Neuron node, using a sigmoid as
        the threshold function.

        returns: number (float or int)
        """
        # sum of all inputs*weights which enter this function
        out = 0

        inputs = self.get_inputs()
        weights = self.get_weights()

        for i in range(len(inputs)):
            out += inputs[i].output() * weights[i].get_value()

        return ( 1.0 / (1.0 + math.exp(-out)) )

    def dOutdX(self, elem):
        # Implement compute_doutdx instead!!
        if self.use_cache:
            # caching optimization, saves previously computed dOutDx.
            if elem not in self.my_doutdx:
                self.my_doutdx[elem] = self.compute_doutdx(elem)
            return self.my_doutdx[elem]
        return self.compute_doutdx(elem)

    def compute_doutdx(self, elem):
        """
        Returns the derivative of this Neuron node, with respect to weight
        elem, calling output() and/or dOutdX() recursively over the inputs.

        elem: an instance of Weight

        returns: number (float/int)
        """
        
        out = self.output()
        # derivative of the threshold, sigmoid, function
        sig_der = out * (1 - out)

        if self.has_weight(elem):
            index = self.my_weights.index(elem)
            sigmoid = self.get_inputs()[index].output()
            d = sig_der*sigmoid
        else:
            d = 0
            for i in range(len(self.get_weights())):
                weight = self.my_weights[i]
                if self.isa_descendant_weight_of(elem, weight):
                    deriv = self.get_inputs()[i].dOutdX(elem)
                    d += weight.get_value() * deriv
            d = d * sig_der
        return d

    def get_weights(self):
        return self.my_weights

    def get_inputs(self):
        return self.my_inputs

    def get_name(self):
        return self.my_name

    def __repr__(self):
        return "Neuron(%s)" %(self.my_name)

class PerformanceElem(DifferentiableElement):
    """
    Representation of a performance computing output node.
    This element contains methods for setting the
    desired output (d) and also computing the final
    performance P of the network.

    This implementation assumes a single output.
    """
    def __init__(self,input,desired_value):
        assert isinstance(input,(Input,Neuron))
        DifferentiableElement.__init__(self)
        self.my_input = input
        self.my_desired_val = desired_value

    def output(self):
        """
        Returns the output of this PerformanceElem node.
        
        returns: number (float/int)
        """
        # P(o) = -0.5 (d - o)**2

        P = -0.5 * (self.my_desired_val - self.my_input.output()) ** 2;

        return P;

    def dOutdX(self, elem):
        """
        Returns the derivative of this PerformanceElem node with respect
        to some weight, given by elem.

        elem: an instance of Weight

        returns: number (int/float)
        """
        # dP/d(w) = dP/do * do/dw # applying the chain rule
        # = (d-o) * o.dOutdX(w)
        return ( self.my_desired_val - self.my_input.output() ) * self.my_input.dOutdX(elem)

    def set_desired(self,new_desired):
        self.my_desired_val = new_desired

    def get_input(self):
        return self.my_input

def alphabetize(x,y):
    if x.get_name()>y.get_name():
        return 1
    return -1

class Network(object):
    def __init__(self,performance_node,neurons):
        self.inputs =  []
        self.weights = []
        self.performance = performance_node
        self.output = performance_node.get_input()
        self.neurons = neurons[:]
        self.neurons.sort(key = cmp_to_key(alphabetize))
        for neuron in self.neurons:
            self.weights.extend(neuron.get_weights())
            for i in neuron.get_inputs():
                if isinstance(i,Input) and not i.get_name()=='i0' and not i in self.inputs:
                    self.inputs.append(i)
        self.weights.reverse()
        self.weights = []
        for n in self.neurons:
            self.weights += n.get_weight_nodes()

    def clear_cache(self):
        for n in self.neurons:
            n.clear_cache()

def seed_random():
    """Seed the random number generator so that random
    numbers are deterministically 'random'"""
    random.seed(0)

def random_weight():
    """Generate a deterministic random weight"""
    # We found that random.randrange(-1,2) to work well emperically 
    # even though it produces randomly 3 integer values -1, 0, and 1.
    return random.randrange(-1, 2)

    # Uncomment the following if you want to try a uniform distribuiton 
    # of random numbers compare and see what the difference is.
    # return random.uniform(-1, 1)

def make_neural_net_basic():
    """
    Constructs a 2-input, 1-output Network with a single neuron.
    This network is used to test your network implementation
    and a guide for constructing more complex networks.

    Naming convention for each of the elements:

    Input: 'i'+ input_number
    Example: 'i1', 'i2', etc.
    Conventions: Start numbering at 1.
                 For the -1 inputs, use 'i0' for everything

    Weight: 'w' + from_identifier + to_identifier
    Examples: 'w1A' for weight from Input i1 to Neuron A
              'wAB' for weight from Neuron A to Neuron B

    Neuron: alphabet_letter
    Convention: Order names by distance to the inputs.
                If equal distant, then order them left to right.
    Example:  'A' is the neuron closest to the inputs.

    All names should be unique.
    You must follow these conventions in order to pass all the tests.
    """
    i0 = Input('i0', -1.0) # this input is immutable
    i1 = Input('i1', 0.0)
    i2 = Input('i2', 0.0)

    w1A = Weight('w1A', 1)
    w2A = Weight('w2A', 1)
    wA  = Weight('wA', 1)

    # Inputs must be in the same order as their associated weights
    A = Neuron('A', [i1,i2,i0], [w1A,w2A,wA])
    P = PerformanceElem(A, 0.0)

    net = Network(P,[A])
    return net

def make_neural_net_two_layer():
    """
    Create a 2-input, 1-output Network with three neurons.
    There should be two neurons at the first level, each receiving both inputs
    Both of the first level neurons should feed into the second layer neuron.

    See 'make_neural_net_basic' for required naming convention for inputs,
    weights, and neurons.
    """
    i0 = Input('i0', -1.0)
    i1 = Input('i1', 0.0)
    i2 = Input('i2', 0.0)

    seed_random()

    wA = Weight('wA', random_weight() )
    wB = Weight('wB', random_weight() )
    w1A = Weight('w1A', random_weight() )
    w1B = Weight('w1B', random_weight() )
    w2A = Weight('w2A', random_weight() )
    w2B = Weight('w2B', random_weight() )

    wAC = Weight('wAC', random_weight() ) 
    wBC = Weight('wBC', random_weight() )
    wC = Weight('wC', random_weight() )

    A = Neuron('A', [i0,i1,i2], [wA,w1A,w2A])
    B = Neuron('B', [i0,i1,i2], [wB,w1B,w2B])

    C = Neuron('C', [i0,A,B], [wC,wAC,wBC])

    P = PerformanceElem(C, 0.0)

    net = Network(P, [A, B, C])
    return net
    # raise NotImplementedError("Implement me!")

def make_neural_net_challenging():
    """
    Design a network that can in-theory solve all 3 problems described in
    the lab instructions.  Your final network should contain
    at most 5 neuron units.

    See 'make_neural_net_basic' for required naming convention for inputs,
    weights, and neurons.
    """
    i0 = Input('i0', -1.0)
    i1 = Input('i1', 0.0)
    i2 = Input('i2', 0.0)

    seed_random()
    w1A = Weight('w1A', random_weight())
    w1B = Weight('w1B', random_weight())
    w1C = Weight('w1C', random_weight())
    w2A = Weight('w2A', random_weight())
    w2B = Weight('w2B', random_weight())
    w2C = Weight('w2C', random_weight())

    wA = Weight('wA', random_weight())
    wB = Weight('wB', random_weight())
    wC = Weight('wC', random_weight())
    wD = Weight('wD', random_weight())

    wAD = Weight('wAD', random_weight())
    wBD = Weight('wBD', random_weight())
    wCD = Weight('wCD', random_weight())

    A = Neuron('A', [i0, i1, i2], [wA, w1A, w2A])
    B = Neuron('B', [i0, i1, i2], [wB, w1B, w2B])
    C = Neuron('C', [i0, i1, i2], [wC, w1C, w2C])
    D = Neuron('D', [i0, A, B, C], [wD, wAD, wBD, wCD])

    P = PerformanceElem(D, 0.0)

    net = Network(P, [A, B, C, D])
    return net

    # raise NotImplementedError("Implement me!")

def make_neural_net_with_weights():
    """
    In this method you are to use the network you designed earlier
    and set pre-determined weights.  Your goal is to set the weights
    to values that will allow the "patchy" problem to converge quickly.
    Your output network should be able to learn the "patchy"
    dataset within 1000 iterations of back-propagation.
    """
    # You can preset weights for the network by completing
    # and uncommenting the init_weights dictionary below.
    #
    
    
   
    init_weights = { 
      'wA': 1,
      'w1A': -1,
      'w2A': 1,
      'wB': -1,
      'w1B': 1,
      'w2B': -1,
      'wC': 1,
      'w1C': -1,
      'w2C': 1,
      'wD': -1,
      'wAD': 1,
      'wBD': -1,
      'wCD': 1,
    }


    return make_net_with_init_weights_from_dict(make_neural_net_challenging,
                                                init_weights)

def make_net_with_init_weights_from_dict(net_fn,init_weights):
    net = net_fn()
    for w in net.weights:
        w.set_value(init_weights[w.get_name()])
    return net

def make_net_with_init_weights_from_list(net_fn,init_weights):
    net = net_fn()
    for i in range(len(net.weights)):
        net.weights[i].set_value(init_weights[i])
    return net


def abs_mean(values):
    """Compute the mean of the absolute values a set of numbers.
    For computing the stopping condition for training neural nets"""
    abs_vals = map(lambda x: abs(x), values)
    total = sum(abs_vals)

    #test
    if(len(list(abs_vals)) != 0):
      return total / float(len(list(abs_vals)))

    return total

def train(network,
          data,      # training data
          rate=1.0,  # learning rate
          target_abs_mean_performance=0.0001,
          max_iterations=10000,
          verbose=False):
    """Run back-propagation training algorithm on a given network.
    with training [data].   The training runs for [max_iterations]
    or until [target_abs_mean_performance] is reached.
    """
    iteration = 0
    while iteration < max_iterations:
        fully_trained = False
        performances = []  # store performance on each data point
        for datum in data:
            # set network inputs
            for i in range(len(network.inputs)):
                network.inputs[i].set_value(datum[i])

            # set network desired output
            network.performance.set_desired(datum[-1])

            # clear cached calculations
            network.clear_cache()

            # compute all the weight updates
            for w in network.weights:
                w.set_next_value(w.get_value() +
                                 rate * network.performance.dOutdX(w))

            # set the new weights
            for w in network.weights:
                w.update()

            # save the performance value
            performances.append(network.performance.output())

            # clear cached calculations
            network.clear_cache()

        # compute the mean performance value
        abs_mean_performance = abs_mean(performances)

        if abs_mean_performance < target_abs_mean_performance:
            if verbose:
                print("iter %d: training complete.\n"\
                      "mean-abs-performance threshold %s reached (%1.6f)"\
                      %(iteration,
                        target_abs_mean_performance,
                        abs_mean_performance))
            break

        iteration += 1
        if iteration % 1000 == 0 and verbose:
            print("iter %d: mean-abs-performance = %1.6f"\
                  %(iteration,
                    abs_mean_performance))


def test(network, data, verbose=False):
    """Test the neural net on some given data."""
    correct = 0
    for datum in data:

        for i in range(len(network.inputs)):
            network.inputs[i].set_value(datum[i])

        # clear cached calculations
        network.clear_cache()
        result = network.output.output()
        network.clear_cache()

        rounded_result = round(result)
        if round(result)==datum[-1]:
            correct+=1
            if verbose:
                print("test(%s) returned: %s => %s [%s]" %(str(datum),
                                                           str(result),
                                                           rounded_result,
                                                           "correct"))
        else:
            if verbose:
                print("test(%s) returned: %s => %s [%s]" %(str(datum),
                                                           str(result),
                                                           rounded_result,
                                                           "wrong"))

    return float(correct)/len(data)


## Neural net data

In [ ]:
#
# Training and Test Data used in the Neural net tester code.
#
"""
1++
0-+
 01
"""
or_data = ((0,0,0),
           (0,1,1),
           (1,0,1),
           (1,1,1),
           (0.25,0,0),
           (0,0.25,0))

or_test_data = ((0.1,0.1,0),
                (0.1,0.9,1),
                (0.9,0.1,1),
                (0.9,0.9,1))
"""
1-+
0--
 01
"""
and_data = ((0,0,0),
            (0,1,0),
            (1,0,0),
            (1,1,1),
            (0.75,1.0,1),
            (1.0,0.75,1))

and_test_data = ((0.1,0.1,0),
                 (0.1,0.9,0),
                 (0.9,0.1,0),
                 (0.9,0.9,1))

"""
1-+
0+-
 01
"""
equal_data = ((0,0,1),
              (0,1,0),
              (1,0,0),
              (1,1,1))

equal_test_data = ((0.1,0.1,1),
                   (0.1,0.9,0),
                   (0.9,0.1,0),
                   (0.9,0.9,1))

"""
1+-
0-+
 01
"""
neq_data = ((0,0,0),
            (0,1,1),
            (1,0,1),
            (1,1,0))

neq_test_data = ((0.1,0.1,0),
                 (0.1,0.9,1),
                 (0.9,0.1,1),
                 (0.9,0.9,0))


"""
3-++-
2-++-
1-++-
0-++-
 0123
"""
vert_band_data = ((0,0,0),
                  (0,1,0),
                  (0,2,0),
                  (0,3,0),
                  (1,0,1),
                  (1,1,1),
                  (1,2,1),
                  (1,3,1),
                  (2,0,1),
                  (2,1,1),
                  (2,2,1),
                  (2,3,1),
                  (3,0,0),
                  (3,1,0),
                  (3,2,0),
                  (3,3,0))

vert_band_test_data = ((0,    1, 0),
                        (0,    2, 0),
                        (0,  1.5, 0),

                        (1.5,  2, 1),
                        (1.5,  5, 1),
                        (1.5,  1, 1),

                        (3,    1, 0),
                        (3,  1.5, 0),
                        (3,    2, 0),

                        (1,  1.5, 1),
                        (1, -1.5, 1),
                        (2,  1.5, 1),
                        (2, -1.5, 1),

                        (4,  0,   0),
                        (4,  4,   0),
                        (-1, 0,   0),
                        (-1, 4,   0))

"""
3----
2++++
1++++
0----
 0123
"""
horiz_band_data = ((0,0,0),
                   (0,1,1),
                   (0,2,1),
                   (0,3,0),
                   (1,0,0),
                   (1,1,1),
                   (1,2,1),
                   (1,3,0),
                   (2,0,0),
                   (2,1,1),
                   (2,2,1),
                   (2,3,0),
                   (3,0,0),
                   (3,1,1),
                   (3,2,1),
                   (3,3,0))

horiz_band_test_data = ((1, 1.5, 1),
                        (2, 1.5, 1),
                        (3, 1.5, 1),
                        (0, 1.5, 1),
                        (4,   0, 0),
                        (4,   4, 0),
                        (-1,  0, 0),
                        (-1,  4, 0))

"""
4--- +
3-- +
2- + -
1 + --
0+ ---
 01234
"""
diag_band_data = ((0,0,1),
                  (1,1,1),
                  (2,2,1),
                  (3,3,1),
                  (4,4,1),
                  (0,4,0),
                  (4,0,0),
                  (0,3,0),
                  (3,0,0),
                  (0,2,0),
                  (2,0,0),
                  (1,4,0),
                  (4,1,0),
                  (1,3,0),
                  (3,1,0),
                  (2,4,0),
                  (4,2,0),
                  )

diag_band_test_data = ((-1,-1,1),
                       (5,  5,1),
                       (-2,-2,1),
                       (6,  6,1),
                       (3.5,3.5,1),
                       (1.5,1.5,1),
                       (4,  0,0),
                       (0,  4,0))

"""
4+++ -
3++ -
2+ - +
1 - ++
0- +++
 01234
"""
idiag_band_data = ((0,0,0),
                   (1,1,0),
                   (2,2,0),
                   (3,3,0),
                   (4,4,0),
                   (0,4,1),
                   (4,0,1),
                   (0,3,1),
                   (3,0,1),
                   (0,2,1),
                   (2,0,1),
                   (1,4,1),
                   (4,1,1),
                   (1,3,1),
                   (3,1,1),
                   (2,4,1),
                   (4,2,1),
                   )

idiag_band_test_data = ((-1,-1,0),
                        (5,  5,0),
                        (-2,-2,0),
                        (6,  6,0),
                        (3.5,3.5,0),
                        (1.5,1.5,0),
                        (4,  0,1),
                        (0,  4,1))


"""
4-----
3-   -
2- + -
1-   -
0-----
 01234
"""
moat_data = ((0,0,0),
             (1,0,0),
             (2,0,0),
             (3,0,0),
             (4,0,0),

             (1,1,0),
             (4,1,0),

             (1,2,0),
             (3,3,1),
             (4,2,0),

             (1,4,0),
             (4,4,0),

             (0,4,0),
             (1,4,0),
             (2,4,0),
             (3,4,0),
             (4,4,0),
             )

moat_test_data = moat_data

"""
4+-
3+-
2+-
1+----
0-++++
 01234
"""
letter_l_data = ((0,0,0),
                 (1,0,1),
                 (2,0,1),
                 (3,0,1),
                 (4,0,1),

                 (1,1,0),
                 (2,1,0),
                 (3,1,0),
                 (4,1,0),

                 (0,2,1),
                 (1,2,0),

                 (0,3,1),
                 (1,3,0),

                 (0,4,1),
                 (1,4,0),
                 )

letter_l_test_data = letter_l_data

"""
4-- ++
3-- ++
2
1++ --
0++ --
 01234
"""
patch_data = ((0,0,1),
              (0,1,1),
              (1,0,1),
              (1,1,1),

              (3,0,0),
              (3,1,0),
              (4,0,0),
              (4,1,0),

              (0,3,0),
              (0,4,0),
              (1,3,0),
              (1,4,0),

              (3,3,1),
              (3,4,1),
              (4,3,1),
              (4,4,1)
              )

patch_test_data = patch_data

simple_data_sets = [("OR", or_data, or_test_data),
                    ("AND", and_data, and_test_data)
                    ]

harder_data_sets = [("EQUAL", equal_data, equal_test_data),
                    ("NOT_EQUAL", neq_data, neq_test_data),
                    ("horizontal-bands", horiz_band_data, horiz_band_test_data),
                    ("vertical-bands", vert_band_data, vert_band_test_data),
                    ("diagonal-band", diag_band_data, diag_band_test_data),
                    ("inverse-diagonal-band", idiag_band_data,
                     idiag_band_test_data)
                    ]

challenging_data_sets = [("moat", moat_data, moat_test_data),
                         ("letter-l", letter_l_data, letter_l_test_data),
                         ]

manual_weight_data_sets = [("patchy", patch_data, patch_test_data)]


all_data_sets = simple_data_sets + harder_data_sets + challenging_data_sets + \
                manual_weight_data_sets


## Neural net tester

In [ ]:
#
# Unit tester for Neural net code.
#
import sys

def main(neural_net_func, data_sets, max_iterations=10000):
    verbose = True
    for name, training_data, test_data in data_sets:
        print("-"*40)
        print("Training on %s data" %(name))
        nn = neural_net_func()
        train(nn, training_data, max_iterations=max_iterations,
              verbose=verbose)
        print("Trained weights:")
        for w in nn.weights:
            print("Weight '%s': %f"%(w.get_name(),w.get_value()))
        print("Testing on %s test-data" %(name))
        result = test(nn, test_data, verbose=verbose)
        print("Accuracy: %f"%(result))

def neural_net_tester(arg=None):
    test_names = ["simple"]
    if arg is not None:
        test_names = arg

    for test_name in test_names:
        if test_name == "simple":
            # these test simple logical configurations
            main(make_neural_net_basic,
                 simple_data_sets)

        elif test_name == "two_layer":
            # these test cases are slightly harder
            main(make_neural_net_two_layer,
                 simple_data_sets + harder_data_sets)

        elif test_name == "challenging":
            # these tests require a more complex architecture.
            main(make_neural_net_challenging, challenging_data_sets)

        elif test_name == "patchy":
            # patchy problem is slightly tricky
            # unless your network gets the right weights.
            # it can quickly get stuck in local maxima.
            main(make_neural_net_challenging, manual_weight_data_sets)

        elif test_name == "weights":
            # if you set the 'right' weights for
            # the patchy problem it can converge very quickly.
            main(make_neural_net_with_weights, manual_weight_data_sets,100)
        else:
            print("unrecognized test name %s" %(test_name))


## Data reader

In [ ]:
"""
A set of utility functions for reading in the data format used by Keith T.
Poole on voteview.com.

You can download additional data that will work with these functions, for
any Congress going back to the 1st, on that site.
"""
import csv
from copy import deepcopy

def legislator_info(legislator):
    district = ''
    if legislator['district'] > 0: district = '-%s' % legislator['district']
    return "%s (%s%s)" % (legislator['name'], legislator['state'], district)

def vote_info(vote):
    if not vote['name']: return vote['number']
    return "%s: %s" % (vote['number'], vote['name'])

def is_interesting(vote):
    return (vote['name'] != '')

def title_case(str):
    chars = list(str)
    chars[0] = chars[0].upper()
    for i in range(1, len(chars)):
        if chars[i-1] not in ' -': chars[i] = chars[i].lower()
    return ''.join(chars)

state_codes = {}
f = open('states.dat')
for line in f:
    state_codes[int(line[0:2])] = title_case(line[6:].strip())
f.close()

party_codes = {}
f = open('party3.dat')
for line in f:
    party_codes[int(line[2:6])] = line[8:].strip()
f.close()

def vote_meaning(n):
    if n in [1, 2, 3]: return 1
    elif n in [4, 5, 6]: return -1
    else: return 0

def read_congress_data(filename):
    """
    Reads a database of Congressional information in the format that comes
    from Keith T. Poole's voteview.com.
    """
    f = open(filename)
    legislators = []
    for line in f:
        line = line.rstrip()
        person = {}
        person['state'] = state_codes[int(line[8:10])]
        person['district'] = int(line[10:12])
        person['party'] = party_codes[int(line[19:23])]
        name = line[25:36].strip()
        person['name'] = title_case(name.replace("  ", ", "))
        person['votes'] = [vote_meaning(int(x)) for x in line[36:]]
        legislators.append(person)
    f.close()
    return legislators

def read_vote_data(filename):
    """
    Reads a CSV file of data on the votes that were taken.
    """
    f = open(filename, encoding='utf-8')
    csv_reader = csv.reader(f)
    votes = []
    for row in csv_reader:
        if row[0] == "date":
            continue # first line with column headers
        vote = {}
        vote['date'] = row[0]
        vote['id'] = str(row[2])
        if vote['id'] == '':
            vote['id'] = str(len(votes))
        vote['number'] = row[3]
        vote['motion'] = row[4]
        vote['name'] = row[6]
        vote['result'] = row[5]
        votes.append(vote)
    f.close()
    return votes

def limit_votes(legislators, votes, n):
    indices = [i for i in range(len(legislators[0]['votes'])-1, -1, -1) if
    is_interesting(votes[i])][:n]

    newleg = []
    for leg in legislators:
        leg = deepcopy(leg)
        leg['votes'] = [leg['votes'][i] for i in indices]
        found_any_votes = False
        for vote in leg['votes']:
            if vote != 0:
                found_any_votes = True
                break
        if found_any_votes: newleg.append(leg)
    newvotes = [votes[i] for i in indices]
    return newleg, newvotes


## Boost

In [ ]:
try:
    set()
except:
    from sets import Set as set
    
import math

class Classifier():
    """
    An abstract class for classification. BaseClassifiers, StandardClassifiers
    and BoostClassifiers inherit from this.
    """
    def __init__(self):
        raise NotImplementedError("This is an abstract class")
    def classify(self, obj):
        raise NotImplementedError()
    def error_rate(self, data, standard):
        """
        Compare this classifier to a StandardClassifier and compute the
        error rate.
        """
        score = 0
        total = len(data)
        for datum in data:
            if self.classify(datum) != standard.classify(datum): score += 1
        return float(score)/total


class StandardClassifier(Classifier):
    """
    A classifier that returns the gold standard value.

    In short, this classifier is one that, by designs, always returns the
    "right answer". Its reason for existence is so that you can compare other
    classifiers to it to test their accuracy.
    """
    def __init__(self, key, value):
        """
        Create a StandardClassifier.

        key: the dictionary key or list index that stores an object's correct
        class.

        value: the class value that will be represented as +1. All other values
        will be represented as -1.
        """
        self.key = key
        self.value = value

    def classify(self, obj):
        """
        If this object's class matches the given value, return +1. Otherwise,
        return -1.
        """
        if obj[self.key] == self.value: return 1
        else: return -1

    def __repr__(self):
        return "<StandardClassifier: %s=%s>" % (self.key, self.value)

# The gold standard classifier we will use throughout is
# standardPartyClassifier, which returns +1 for Republican or -1 for any other
# party. The other party might be Democrat, Independent, or (for the 1796 data
# set) Federalist.

standardPartyClassifier = StandardClassifier('party', 'Republican')


class BaseVoteClassifier(Classifier):
    """
    A simplistic classifier that classifies a legislator based on a single
    vote.
    """
    def __init__(self, index, value, votelist):
        """
        Make a base classifier that classifies a dictionary representing
        a legislator, based on one of their votes.

        index: the vote number to look up.
        value: the value to check for. The classifier will return 1 if it
               matches, and -1 if it doesn't.
        votelist: a list of dictionaries that the classifier can use to
                  look up the meanings of the votes.
        """
        self.index = index
        self.value = value
        self.votelist = votelist

    def classify(self, legislator):
        if legislator['votes'][self.index] == self.value: return 1
        else: return -1

    def __str__(self):
        vote = vote_info(self.votelist[self.index])
        if self.value == 1: direction = 'YES'
        else: direction = 'NO'
        return "%s on %s" % (direction, vote)
    def __repr__(self):
        return "<BaseVoteClassifier: %s>" % self


def sigmoid(x):
    return (float(1)/(1+math.exp(-x)))

def error_to_alpha(error_rate):
    """
    Given an error rate, convert it to an alpha value -- that is, a weight to
    assign to a base classifier. Low error rates get high alpha values.
    """
    doubt = 1e-4
    error_rate = min(max(error_rate, doubt), 1-doubt)
    return (math.log(1-error_rate) - math.log(error_rate)) * 0.5

class BoostClassifier(Classifier):
    """
    A classifier that learns by composing several base classifiers using the
    AdaBoost algorithm.
    """
    def __init__(self, base_classifiers, data, standard):
        """
        Create a BoostClassifier.

        base_classifiers: A list of all possible base classifiers to use. Note
        that you automatically get the opposites of these classifiers as well.

        data: the list of data points. These should be classifiable both by the
        base classifiers and by the gold standard classifier.

        standard: the "gold standard" classifier that always returns the
        correct class for a data point.
        """
        self.base_classifiers = base_classifiers
        self.data = data 
        self.data_weights = [1.0/len(data) for d in data]
        self.classifiers = []
        self.standard = standard

    def classify(self, obj):
        """
        Once the boost classifier is trained, this function will use the
        weighted combination of the base classifiers it learned to output a
        final value.

        It should return a class of 1 if the weighted total is positive, and -1
        otherwise.

        obj: a data point to classify

        returns: int (+1 or -1)
        """
        result = 0
        for classifier, weight in self.classifiers:
            result += classifier.classify(obj) * weight
        if result >= 0:
            return 1
        else:
            return -1

    def best_classifier(self):
        """
        Returns the best base classifier for the current weights, along
        with its error rate.

        NOTE: the "best" classifier is the one that is the best at
        distinguishing the two sets of data points. We're not requiring,
        however, that it is the best at distinguishing them in the direction
        you asked for. That is, a very high error rate like 0.999 is just as
        good as a very low error rate like 0.001!

        Why are we doing this? Because it essentially lets us invert any
        base classifier. The mathematics of boosting means that it will
        automatically count classifiers with error rates _above_ 0.5 as having
        a negative weight, making them act just like their opposite classifier
        which would have a low error rate.

        As an example, this means that if one possible base classifier is
        "voted YES on legalizing ferrets", this could also recognize a class
        that "voted NO or abstained on legalizing ferrets" just by giving you
        the YES classifier with a negative alpha (= a high error rate).
        """
        best_error = 0.5
        best_classifier = None
        for classifier in set(self.base_classifiers) - set(self.classifiers):
            error = 0.0
            for datum, dweight in zip(self.data, self.data_weights):
                if classifier.classify(datum) != self.standard.classify(datum):
                    error += dweight
            if abs(error-0.5) > abs(best_error-0.5):
                best_error = error
                best_classifier = classifier
        if best_classifier is None:
            # none of the classifiers work?
            best_classifier = self.base_classifiers[0]
        return (best_classifier, best_error)

    def train(self, steps=10, verbose=False):
        """
        Run the AdaBoost algorithm for a specified number of steps.
        """
        for i in range(steps): self.step(verbose=verbose)

    def reset(self):
        """
        Reset the classifier to its untrained state.
        """
        self.data_weights = [1.0/len(self.data) for d in self.data]
        self.classifiers = []

    def renormalize_weights(self):
        """
        Ensure that the weights of the data points add up to 1.

        This should be called at every step. Even if your algorithm inherently
        makes the points add up to 1, you may eventually need to correct
        for floating-point drift.
        """
        total = sum(self.data_weights)
        self.data_weights = [w/total for w in self.data_weights]

    def step(self, verbose=False):
        """
        Run one step of boosting:

        * Renormalize the weights
        * Find the classifier that is best at distinguishing the classes
          given these weights
        * Update the weights based on the result of the classifier
        """
        self.renormalize_weights()
        best_classifier, best_error = self.best_classifier()
        if verbose:
            print("[error=%4.4f]" % best_error, best_classifier)
        self.update_weights(best_error, best_classifier)
        self.classifiers.append((best_classifier, error_to_alpha(best_error)))

    def update_weights(self, best_error, best_classifier):
        """
        Follows the boosting algorithm to update the weights of the data points.

        best_error: number (int/float) that is the error of the best classifier
        best_classifier: Classifier instance which best classifies the data

        returns: Nothing (only updates self.data_weights)
        """
  #       [FORMULA]:
  #  - [correctly classified point]: 1/2 * [ 1 / (1 - [error rate]) ] * [weight.old]
  #  - [wrongly   classified point]: 1/2 * [ 1 /      [error rate]  ] * [weight.old]
  # - This is done especially to highlight the [misclassified] points
  # - We want [wrongly classified point] to have their [weight] increased,
  #  so they are highlighted -> [correct points] have their [weight] lowered
  
        for i in range(len(self.data_weights)):
            if best_classifier.classify(self.data[i]) == self.standard.classify(self.data[i]):
                self.data_weights[i] = 0.5*(self.data_weights[i])/(1-best_error)
            else:
                self.data_weights[i] = 0.5*(self.data_weights[i])/best_error

    def __str__(self):
        classifier_part = '\n'.join(["%4.4f: %s" % (weight, c) for c, weight in
        self.classifiers])
        return "Boosting classifier:\n"+classifier_part

    def __repr__(self):
        return "<Boosting classifier: %r>" % self.classifiers

def make_vote_classifiers(votelist):
    """
    Given a list of votes, make two BaseVoteClassifiers for each vote
    and return them all in a list.
    """
    classifiers = []
    for index in range(len(votelist)):
        for value in (1, -1):
            classifiers.append(BaseVoteClassifier(index, value, votelist))
    return classifiers

## To be implemented

In [ ]:
# Senate and House Data
# These should be familiar by now.
senate_people = read_congress_data('S110.ord')
senate_votes = read_vote_data('S110desc.csv')

house_people = read_congress_data('H110.ord')
house_votes = read_vote_data('H110desc.csv')

last_senate_people = read_congress_data('S109.ord')
last_senate_votes = read_vote_data('S109desc.csv')

house_1796 = read_congress_data('H004.ord')
house_1796_votes = read_vote_data('H004desc.csv')


# The first step is to complete the boosting code in boost.py. None of the
# following steps will work without it.
#
# Once you've done that, the following line should make a boost classifier
# that learns about the 4th House of Representatives (1795-1796).

boost_1796 = BoostClassifier(make_vote_classifiers(house_1796_votes),
                             house_1796, standardPartyClassifier)

# You will need to train it, however. You can change the number of steps here.
boost_1796.train(20)

# Once you have run your boosting classifier for a sufficient number of steps
# on the 4th House of Representatives data, it should tell you how it believes
# Republicans and Federalists generally voted on a range of issues. Which way
# does it predict a Republican would vote on the amendment to require
# "newspapers to be sufficiently dried before mailing"? ('yes' or 'no')

republican_newspaper_vote = 'no'

# In the 4th House of Representatives, which five representatives were
# misclassified the most while training your boost classifier?
#
# You should answer this question by defining the following function.
# It should return five names of legislators, in the format that comes from
# the legislator_info function. The tests will check the function, not just
# its output in this case.

def most_misclassified(classifier, n=5):
    """
    Given a trained boosting classifier, return the n data points that were
    misclassified the most (based on their final weights). The
    most-misclassified datum should be at the beginning of the list.

	You will need to use the "legislator_info(datum)" function to put your
	output in the correct format.

	classifier: instance of Classifier -- used to classify the data
	n: int -- the number of most-misclassified data points to return

	returns: list of data points (each passed through legislator_info) that were
			 misclassified most often
    """
   
    # weights = [(classifier.data_weights[i], i) for i in range(len(classifier.data_weights))]
    # output = []

    # s = sorted(weights, key = lambda k : k[0])
    # s.reverse()
    # for i in range(n):
    #     index = s[i][1]
    #     output.append(legislator_info(classifier.data[index]))
    # return output 
   

    weights = [(classifier.data_weights[i], i) for i in range(len(classifier.data_weights))]

    sortedWeights = sorted(weights, key=lambda weight : weight[0] )
    sortedWeights.reverse()

    output = [ legislator_info(classifier.data[index]) for index in [ x[1] for x in sortedWeights] ]
    
    final = output[:n]

    return final

# The following line is used by the tester; please leave it in place!
most_misclassified_boost_1796 = lambda n: most_misclassified(boost_1796, n)

# print(most_misclassified_boost_1796(5))

# Now train a similar classifier on the 110th Senate (2007-2008).
# How does it say a Republican would vote on Cardin Amdt No. 3930; To modify
# the sunset provision (whatever that is)?

boost = BoostClassifier(make_vote_classifiers(senate_votes), senate_people,
  standardPartyClassifier)
boost.train(20)
republican_sunset_vote = 'no'

# Which five Senators are the most misclassified after training your
# classifier? (Again, the tester will test the function, not the answer you
# print out here.)

# The following line is used by the tester; please leave it in place!
most_misclassified_boost = lambda n: most_misclassified(boost, n)

# print(most_misclassified_boost(5))




## The following code is used by the tester; please leave it in place!
def classifier_tester(classifier_name, data_set):
    """ Test a particular classifier, verify that it improves every step over 20 steps """
    return list(classifier_tester_helper(classifier_name, data_set))

def classifier_tester_helper(classifier_name, data_set):
    if classifier_name in globals():
        classifier = globals()[classifier_name]
        data = globals()[data_set]
        if isinstance(classifier, Classifier):
            original_classifier_count = len(classifier.classifiers)
            classifier.reset()
            for x in range(20):
                classifier.step()
                yield classifier.error_rate(data, standardPartyClassifier)

            classifier.reset()
            classifier.train(original_classifier_count)
            return
    raise Exception("Error: Classifier %s doesn't exist!, can't test it" % classifier_name)

def neural_net_tester(network_maker_func,
                      train_dataset_name,
                      test_dataset_name,
                      iterations):
    """Test a neural net making function on a named dataset"""
    seed_random()
    network_maker_func = globals()[network_maker_func]
    train_dataset = globals()[train_dataset_name]
    test_dataset = globals()[test_dataset_name]
    nn = network_maker_func()

    train(nn, train_dataset, max_iterations=iterations)
    result = test(nn, test_dataset)
    return result

def neural_net_size_tester(network_maker_func):
    """Test a neural net size"""
    network_maker_func = globals()[network_maker_func]
    nn = network_maker_func()
    return len(nn.neurons)


## Tester

In [ ]:
from xmlrpc import client
import traceback
import sys
import os
import tarfile

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO


# This is a skeleton for what the tester should do. Ideally, this module
# would be imported in the pset and run as its main function.

# We need the following rpc functions. (They generally take username and
# password, but you could adjust this for whatever security system.)
#
# tester.submit_code(username, password, pset, studentcode)
#   'pset' is a string such as 'ps0'. studentcode is a string containing
#   the contents of the corresponding file, ps0.py. This stores the code on
#   the server so we can check it later for cheating, and is a prerequisite
#   to the tester returning a grade.
#
# tester.get_tests(pset)
#   returns a list of tuples of the form (INDEX, TYPE, NAME, ARGS):
#     INDEX is a unique integer that identifies the test.
#     TYPE should be one of either 'VALUE' or 'FUNCTION'.
#     If TYPE is 'VALUE', ARGS is ignored, and NAME is the name of a
#     variable to return for this test.  The variable must be an attribute
#     of the lab module.
#     If TYPE is 'FUNCTION', NAME is the name of a function in the lab module
#     whose return value should be the answer to this test, and ARGS is a
#     tuple containing arguments for the function.
#
# tester.send_answer(username, password, pset, index, answer)
#   Sends <answer> as the answer to test case <index> (0-numbered) in the pset
#   named <pset>. Returns whether the answer was correct, and an expected
#   value.
#
# tester.status(username, password, pset)
#   A string that includes the official score for this user on this pset.
#   If a part is missing (like the code), it should say so.

# Because I haven't written anything on the server side, test_online has never
# been tested.

def test_summary(dispindex, ntests, testname):
    return "Test %d/%d (%s)" % (dispindex, ntests, testname)

tests = []

def show_result(testsummary, testcode, correct, got, expected, verbosity):
    """ Pretty-print test results """
    if correct:
        if verbosity > 0:
            print("%s: Correct." % testsummary)
        if verbosity > 1:
            print('\t', testcode)
            print('')
    else:
        print("%s: Incorrect." % testsummary)
        print('\t', testcode)
        print("Got:     ", got)
        print("Expected:", expected)

def show_exception(testsummary, testcode):
    """ Pretty-print exceptions (including tracebacks) """
    print("%s: Error." % testsummary)
    print("While running the following test case:")
    print('\t', testcode)
    print("Your code encountered the following error:")
    traceback.print_exc()
    print('')


def get_lab_module():
    # Try the easy way first
    try:
        from tests import lab_number
    except ImportError:
        lab_number = None

    if lab_number != None:
        lab = __import__('lab%s' % lab_number)
        return lab

    lab = None

    for labnum in range(10):
        try:
            lab = __import__('lab%s' % labnum)
        except ImportError:
            pass

    if lab == None:
        raise ImportError("Cannot find your lab; or, error importing it.  Try loading it by running 'python labN.py' (for the appropriate value of 'N').")

    if not hasattr(lab, "LAB_NUMBER"):
        lab.LAB_NUMBER = labnum

    return lab

def type_decode(arg, lab):
    """
    XMLRPC can only pass a very limited collection of types.
    Frequently, we want to pass a subclass of 'list' in as a test argument.
    We do that by converting the sub-type into a regular list of the form:
    [ 'TYPE', (data) ] (ie., AND(['x','y','z']) becomes ['AND','x','y','z']).
    This function assumes that TYPE is a valid attr of 'lab' and that TYPE's
    constructor takes a list as an argument; it uses that to reconstruct the
    original data type.
    """
    if isinstance(arg, list) and len(arg) >= 1: # We'll leave tuples reserved for some other future magic
        try:
            mytype = arg[0]
            data = arg[1:]
            return getattr(lab, mytype)([ type_decode(x, lab) for x in data ])
        except AttributeError:
            return [ type_decode(x, lab) for x in arg ]
        except TypeError:
            return [ type_decode(x, lab) for x in arg ]
    else:
        return arg


def type_encode(arg):
    """
    Encode trees as lists in a way that can be decoded by 'type_decode'
    """
    if isinstance(arg, list) and not type(arg) in (list,tuple):
        return [ arg.__class__.__name__ ] + [ type_encode(x) for x in arg ]
    elif hasattr(arg, '__class__') and arg.__class__.__name__ == 'IF':
        return [ 'IF', type_encode(arg._conditional), type_encode(arg._action), type_encode(arg._delete_clause) ]
    else:
        return arg


def run_test(test, lab):
    """
    Takes a 'test' tuple as provided by the online tester
    (or generated by the offline tester) and executes that test,
    returning whatever output is expected (the variable that's being
    queried, the output of the function being called, etc)

    'lab' (the argument) is the module containing the lab code.

    'test' tuples are in the following format:
      'id': A unique integer identifying the test
      'type': One of 'VALUE', 'FUNCTION', 'MULTIFUNCTION', or 'FUNCTION_ENCODED_ARGS'
      'attr_name': The name of the attribute in the 'lab' module
      'args': a list of the arguments to be passed to the function; [] if no args.
      For 'MULTIFUNCTION's, a list of lists of arguments to be passed in
    """
    id, mytype, attr_name, args = test

    attr = getattr(lab, attr_name)

    if mytype == 'VALUE':
        return attr
    elif mytype == 'FUNCTION':
        return apply(attr, args)
    elif mytype == 'MULTIFUNCTION':
        return [ run_test( (id, 'FUNCTION', attr_name, FN), lab) for FN in args ]
    elif mytype == 'FUNCTION_ENCODED_ARGS':
        return run_test( (id, 'FUNCTION', attr_name, type_decode(args, lab)), lab )
    else:
        raise Exception("Test Error: Unknown TYPE '%s'.  Please make sure you have downloaded the latest version of the tester script.  If you continue to see this error, contact a TA.")


def test_offline(verbosity=1):
    """ Run the unit tests in 'tests.py' """
    import tests as tests_module

#    tests = [ (x[:-8],
#               getattr(tests_module, x),
#               getattr(tests_module, "%s_testanswer" % x[:-8]),
#               getattr(tests_module, "%s_expected" % x[:-8]),
#               "_".join(x[:-8].split('_')[:-1]))
#              for x in tests_module.__dict__.keys() if x[-8:] == "_getargs" ]

#    tests = tests_module.get_tests()
    global tests 

    ntests = len(tests)
    ncorrect = 0

    for index, (testname, getargs, testanswer, expected, fn_name, type) in enumerate(tests):
        dispindex = index+1
        summary = test_summary(dispindex, ntests, fn_name)

        try:
            if callable(getargs):
                getargs = getargs()

            if type == 'FUNCTION':
                answer = fn_name(*getargs)
            elif type == 'VALUE':
                answer = fn_name
            else:
                answer = [ FN(*getargs) for FN in getargs ]#run_test((index, type, fn_name, getargs), get_lab_module())
        except NotImplementedError:
            print("%d: (%s: Function not yet implemented, NotImplementedError raised)" % (index, testname))
            continue
        except Exception:
            show_exception(summary, testname)
            continue

        correct = testanswer(answer, original_val = getargs)
        show_result(summary, testname, correct, answer, expected, verbosity)
        if correct: ncorrect += 1

    print("Passed %d of %d tests." % (ncorrect, ntests))
    tests = []
    return (ncorrect == ntests)



def get_target_upload_filedir():
    """ Get, via user prompting, the directory containing the current lab """
    cwd = os.getcwd() # Get current directory.  Play nice with Unicode pathnames, just in case.

    print("Please specify the directory containing your lab.")
    print("Note that all files from this directory will be uploaded!")
    print("Labs should not contain large amounts of data; very-large")
    print("files will fail to upload.")
    print('')
    print("The default path is '%s'" % cwd)
    target_dir = raw_input("[%s] >>> " % cwd)

    target_dir = target_dir.strip()
    if target_dir == '':
        target_dir = cwd

    print("Ok, using '%s'." % target_dir)

    return target_dir

def get_tarball_data(target_dir, filename):
    """ Return a binary String containing the binary data for a tarball of the specified directory """
    data = StringIO()
    file = tarfile.open(filename, "w|bz2", data)

    print("Preparing the lab directory for transmission...")

    file.add(target_dir+"/lab5.py")
    file.add(target_dir+"/neural_net.py")
    file.add(target_dir+"/boost.py")
    file.add(target_dir+"/key.py")

    print("Done.")
    print('')
    print("The following files have been added:")

    for f in file.getmembers():
        print(f.name)

    file.close()

    return data.getvalue()


def test_online(verbosity=1):
    """ Run online unit tests.  Run them against the 6.034 server via XMLRPC. """
    lab = get_lab_module()

    try:
        server = xmlrpclib.Server(server_url, allow_none=True)
        print("Getting tests:", (username, password, lab.__name__))
        tests = server.get_tests(username, password, lab.__name__)
        print("*** TESTS:")
        print(tests)

    except NotImplementedError: # Solaris Athena doesn't seem to support HTTPS
        print("Your version of Python doesn't seem to support HTTPS, for")
        print("secure test submission.  Would you like to downgrade to HTTP?")
        print("(note that this could theoretically allow a hacker with access")
        print("to your local network to find your 6.034 password)")
        answer = raw_input("(Y/n) >>> ")
        if len(answer) == 0 or answer[0] in "Yy":
            server = xmlrpclib.Server(server_url.replace("https", "http"))
            tests = server.get_tests(username, password, lab.__name__)
        else:
            print("Ok, not running your tests.")
            print("Please try again on another computer.")
            print("Linux Athena computers are known to support HTTPS,")
            print("if you use the version of Python in the 'python' locker.")
            sys.exit(0)

    ntests = len(tests)
    ncorrect = 0

    lab = get_lab_module()

    target_dir = get_target_upload_filedir()

    tarball_data = get_tarball_data(target_dir, "lab%s.tar.bz2" % lab.LAB_NUMBER)

    print("Submitting to the 6.034 Webserver...")

    server.submit_code(username, password, lab.__name__, xmlrpclib.Binary(tarball_data))

    print("Done submitting code.")
    print("Running test cases...")

    for index, testcode in enumerate(tests):
        dispindex = index+1
        summary = test_summary(dispindex, ntests, testcode)

        try:
            answer = run_test(testcode, get_lab_module())
        except Exception:
            show_exception(summary, testcode)
            continue

        correct, expected = server.send_answer(username, password, lab.__name__, testcode[0], type_encode(answer))
        show_result(summary, testcode, correct, answer, expected, verbosity)
        if correct: ncorrect += 1

    response = server.status(username, password, lab.__name__)
    print(response)


def make_test_counter_decorator():
    #tests = []
    def make_test(getargs, testanswer, expected_val, name = None, type = 'FUNCTION'):
        if name != None:
            getargs_name = name
        elif not callable(getargs):
            getargs_name = "_".join(getargs[:-8].split('_')[:-1])
            getargs = lambda: getargs
        else:
            getargs_name = "_".join(getargs.__name__[:-8].split('_')[:-1])

        tests.append( ( getargs_name,
                        getargs,
                        testanswer,
                        expected_val,
                        getargs_name,
                        type ) )

    def get_tests():
        return tests

    return make_test, get_tests


make_test, get_tests = make_test_counter_decorator()


## Tests

In [ ]:
message = 'your trained neural-net on %s data must test with an accuracy of %1.3f'
expected_accuracy = 1.0

def neural_net_test_testanswer(val, original_val = None):
    return abs(val - expected_accuracy) < 0.01

network_maker_func = "make_neural_net_two_layer"
network_min_size = 3
max_iterations = 10000

def neural_net_size_testanswer(val, original_val = None):
    return val <= network_min_size

make_test(type = 'FUNCTION',
          getargs = lambda: [network_maker_func],
          testanswer = neural_net_size_testanswer,
          expected_val = "your network must have <= %d neural units"\
          %(network_min_size),
          name = neural_net_size_tester
          )

make_test(type = 'FUNCTION',
          getargs = lambda: [network_maker_func,
                             'and_data',
                             'and_test_data',
                             max_iterations],
          testanswer = neural_net_test_testanswer,
          expected_val = message %("AND", expected_accuracy),
          name = neural_net_tester
          )

make_test(type = 'FUNCTION',
          getargs = lambda: [network_maker_func,
                             'or_data',
                             'or_test_data',
                             max_iterations],
          testanswer = neural_net_test_testanswer,
          expected_val = message %("OR", expected_accuracy),
          name = neural_net_tester
          )

make_test(type = 'FUNCTION',
          getargs = lambda: [network_maker_func,
                             'neq_data',
                             'neq_test_data',
                             max_iterations],
          testanswer = neural_net_test_testanswer,
          expected_val = message %("XOR", expected_accuracy),
          name = neural_net_tester
          )

make_test(type = 'FUNCTION',
          getargs = lambda: [network_maker_func,
                             'equal_data',
                             'equal_test_data',
                             max_iterations],
          testanswer = neural_net_test_testanswer,
          expected_val = message %("EQUAL", expected_accuracy),
          name = neural_net_tester
          )

make_test(type = 'FUNCTION',
          getargs = lambda: [network_maker_func,
                             'diag_band_data',
                             'diag_band_test_data',
                             max_iterations],
          testanswer = neural_net_test_testanswer,
          expected_val = message %("diagonal_band", expected_accuracy),
          name = neural_net_tester
          )

challenging_network_maker_func = "make_neural_net_challenging"
challenging_network_min_size = 5

def challenging_neural_net_size_testanswer(val, original_val = None):
    return val <= challenging_network_min_size

make_test(type = 'FUNCTION',
          getargs = lambda: [challenging_network_maker_func],
          testanswer = challenging_neural_net_size_testanswer,
          expected_val = "your network must have <= %d neural units"\
          %(network_min_size),
          name = neural_net_size_tester
          )

make_test(type = 'FUNCTION',
          getargs = lambda: [challenging_network_maker_func,
                             'letter_l_data',
                             'letter_l_test_data',
                             max_iterations],
          testanswer = neural_net_test_testanswer,
          expected_val = message %("letter-l", expected_accuracy),
          name = neural_net_tester
          )

manual_weights_network_maker_func = "make_neural_net_with_weights"
manual_weights_network_max_iterations = 3000

make_test(type = 'FUNCTION',
          getargs = lambda: [manual_weights_network_maker_func,
                             'patch_data',
                             'patch_test_data',
                             manual_weights_network_max_iterations],
          testanswer = neural_net_test_testanswer,
          expected_val = message %("patchy", expected_accuracy),
          name = neural_net_tester
          )

republican_newspaper_vote_getargs = republican_newspaper_vote

def republican_newspaper_vote_testanswer(val, original_val = None):
    return ( val in ('no', 'No', 'nay', 'Nay', 'NO') )

make_test(type = 'VALUE',
          getargs = republican_newspaper_vote_getargs,
          testanswer = republican_newspaper_vote_testanswer,
          expected_val = "no",
          name = republican_newspaper_vote_getargs
          )

def classifier_tester_1_getargs():
    return [ 'boost_1796', 'house_1796' ]

def classifier_tester_1_testanswer(val, original_val = None):
    [0.077586206896551727, 0.077586206896551727, 0.034482758620689655,
    0.051724137931034482, 0.025862068965517241, 0.025862068965517241, 0.0,
    0.0086206896551724137, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
    0.0, 0.0]
    return (abs(val[0]-.077586 < 0.0001) and val[19] < 0.0001)

make_test(type = 'FUNCTION',
          getargs = classifier_tester_1_getargs,
          testanswer = classifier_tester_1_testanswer,
          expected_val = "A classifier that improves as much as would be expected by boosting, over 20 steps",
          name = classifier_tester
          )

def most_misclassified_boost_1796_getargs():
    return [ 5 ]

def most_misclassified_boost_1796_testanswer(val, original_val = None):
    return ( len(val) == 5 and 'Dayton (New Jersey-98)' in val )

make_test(type = 'FUNCTION',
          getargs = most_misclassified_boost_1796_getargs,
          testanswer = most_misclassified_boost_1796_testanswer,
          expected_val = "Five hard-to-classify Congressmen including Dayton",
          name = most_misclassified_boost_1796
          )


republican_sunset_vote_getargs = republican_sunset_vote

def republican_sunset_vote_testanswer(val, original_val = None):
    return ( val in ('no', 'No', 'nay', 'Nay') )

make_test(type = 'VALUE',
          getargs = republican_sunset_vote_getargs,
          testanswer = republican_sunset_vote_testanswer,
          expected_val = "'no'",
          name = republican_sunset_vote_getargs
          )


def most_misclassified_boost_getargs():
    return [ 5 ]

def most_misclassified_boost_testanswer(val, original_val = None):
    return len(val) == 5

make_test(type = 'FUNCTION',
          getargs = most_misclassified_boost_getargs,
          testanswer = most_misclassified_boost_testanswer,
          expected_val = "Five hard-to-classify Senators",
          name = most_misclassified_boost
          )


def classifier_tester_2_getargs():
    return [ 'boost', 'senate_people' ]

def classifier_tester_2_testanswer(val, original_val = None):
    if abs(val[0]-.0098) > .0001: return False

    for x in range(10,len(val)):
        if val[x] > .0001:
            return False

    return True

make_test(type = 'FUNCTION',
          getargs = classifier_tester_2_getargs,
          testanswer = classifier_tester_2_testanswer,
          expected_val = "A classifier that improves as much as would be expected by boosting, over 20 steps",
          name = classifier_tester
          )

test_offline()

Test 1/15 (<function neural_net_size_tester at 0x7fc0728e6e18>): Correct.
Test 2/15 (<function neural_net_tester at 0x7fc07a69c2f0>): Correct.
Test 3/15 (<function neural_net_tester at 0x7fc07a69c2f0>): Correct.
Test 4/15 (<function neural_net_tester at 0x7fc07a69c2f0>): Correct.
Test 5/15 (<function neural_net_tester at 0x7fc07a69c2f0>): Correct.
Test 6/15 (<function neural_net_tester at 0x7fc07a69c2f0>): Correct.
Test 7/15 (<function neural_net_size_tester at 0x7fc0728e6e18>): Correct.
Test 8/15 (<function neural_net_tester at 0x7fc07a69c2f0>): Correct.
Test 9/15 (<function neural_net_tester at 0x7fc07a69c2f0>): Correct.
Test 10/15 (no): Correct.
Test 11/15 (<function classifier_tester at 0x7fc07a9d89d8>): Correct.
Test 12/15 (<function <lambda> at 0x7fc07acdfc80>): Correct.
Test 13/15 (no): Correct.
Test 14/15 (<function <lambda> at 0x7fc0728e6d90>): Correct.
Test 15/15 (<function classifier_tester at 0x7fc07a9d89d8>): Correct.
Passed 15 of 15 tests.


True